# Shift Cipher

In [1]:
import string

In [2]:
class ShiftCipher:
    plaintext_space = set(string.ascii_lowercase)
    
    def __init__(self, key: int):
        self.key = key % len(self.plaintext_space)
        
    @staticmethod
    def shift(ch: str, key: int) -> str:
        assert len(ch) == 1
        assert ch in ShiftCipher.plaintext_space
        offset = ord(ch) - ord('a')
        offset = (offset + key) % len(ShiftCipher.plaintext_space)
        return chr(ord('a') + offset)
    
    def encrypt(self, plaintext: str) -> str:
        out = []
        for ch in plaintext:
            out.append(self.shift(ch, self.key))
        
        return "".join(out)
    
    def decrypt(self, ciphertext: str) -> str:
        out = []
        for ch in ciphertext:
            out.append(self.shift(ch, len(self.plaintext_space) - self.key))
        
        return "".join(out)

In [3]:
def get_freq(text: str) -> dict:
    freq = {}
    for ch in text:
        freq[ch] = freq.get(ch, 0) + 1
    
    return freq

## Largest probability match

In [4]:
def get_key_1(ciphertext: str) -> int:
    assert all(ch in ShiftCipher.plaintext_space for ch in ciphertext)
    
    freq = get_freq(ciphertext)
    
    e = max(freq, key = lambda x: freq[x])
    return (ord(e) - ord('e')) % len(ShiftCipher.plaintext_space)

### Assignment 1 Q1

In [5]:
# ciphertext = input("Enter ciphertext to decrypt: ")
ciphertext = "ylkspwzhyluvazvylkhzaolzahpulkzavulzrpzzlkifaollunspzoklhkrpukulzzvmdvvlkhukdvvlyzlltzzohtlavaolpysvclwbylvsvclfvbylflzsvzlsbyldolupilovsklflzispuklkputfzalhk"
print(f"{ciphertext = }")

# simple largest frequency works here
key = get_key_1(ciphertext)
print(f"{key = }")

cipher = ShiftCipher(key)
decrypted_plaintext = cipher.decrypt(ciphertext)
print(f"{decrypted_plaintext = }")

ciphertext = 'ylkspwzhyluvazvylkhzaolzahpulkzavulzrpzzlkifaollunspzoklhkrpukulzzvmdvvlkhukdvvlyzlltzzohtlavaolpysvclwbylvsvclfvbylflzsvzlsbyldolupilovsklflzispuklkputfzalhk'
key = 7
decrypted_plaintext = 'redlipsarenotsoredasthestainedstoneskissedbytheenglishdeadkindnessofwooedandwooerseemsshametotheirlovepureoloveyoureyesloselurewhenibeholdeyesblindedinmystead'


## Closest to Actual Language Probabilites

In [6]:
def get_key_2(ciphertext: str) -> int:
    freq = get_freq(ciphertext)
    for k in freq:
        freq[k] /= len(ciphertext)
    # print(f"{freq = }")
    
    # http://pi.math.cornell.edu/~mec/2003-2004/cryptography/subs/frequencies.html
    english_freq = {
        "e": 12.02,
        "t": 9.10,
        "a": 8.12,
        "o": 7.68,
        "i": 7.31,
        "n": 6.95,
        "s": 6.28,
        "r": 6.02,
        "h": 5.92,
        "d": 4.32,
        "l": 3.98,
        "u": 2.88,
        "c": 2.71,
        "m": 2.61,
        "f": 2.30,
        "y": 2.11,
        "w": 2.09,
        "g": 2.03,
        "p": 1.82,
        "b": 1.49,
        "v": 1.11,
        "k": 0.69,
        "x": 0.17,
        "q": 0.11,
        "j": 0.10,
        "z": 0.07,
    }
    
    def dist(f1: dict, f2: dict, key: int) -> float:
        out = 0.0
        for ch in ShiftCipher.plaintext_space:
            out += pow(f1.get(ShiftCipher.shift(ch, key), 0) - f2.get(ch, 0), 2)

        return out
    
    dists = [(possible_key, dist(freq, english_freq, possible_key)) for possible_key in range(0, len(ShiftCipher.plaintext_space)) ]
    # print(f"{dists = }")

    key = min(dists, key=lambda x: x[1])[0]
    
    return key


### Assignment 1 Q2

In [7]:
# ciphertext = input("Enter ciphertext to decrypt: ")
ciphertext = "eaddagfkgxtstawkosluzafylzwkcawktwddawkkogddwfoalztayjgmfvwqwkgfbwkkgjwjgsvdgfytsetggzmlkfghdsuwlgkzaltmlksfvuzsffwdjmlkeaddagfkgxxslzwjkafjsafeaddagfkgxeglzwjkafhsafeaddagfkgxtjglzwjkafogweaddagfkgxkaklwjkfgozwjwlgyg"
print(f"{ciphertext = }")

# simple method doesn't work, need full frequency analysis
# key = get_key_1(ciphertext)
key = get_key_2(ciphertext)
print(f"{key = }")                                                                                  

cipher = ShiftCipher(key)
decrypted_plaintext = cipher.decrypt(ciphertext)
print(f"{decrypted_plaintext = }")

ciphertext = 'eaddagfkgxtstawkosluzafylzwkcawktwddawkkogddwfoalztayjgmfvwqwkgfbwkkgjwjgsvdgfytsetggzmlkfghdsuwlgkzaltmlksfvuzsffwdjmlkeaddagfkgxxslzwjkafjsafeaddagfkgxeglzwjkafhsafeaddagfkgxtjglzwjkafogweaddagfkgxkaklwjkfgozwjwlgyg'
key = 18
decrypted_plaintext = 'millionsofbabieswatchingtheskiesbelliesswollenwithbigroundeyesonjessoreroadlongbamboohutsnoplacetoshitbutsandchannelrutsmillionsoffathersinrainmillionsofmothersinpainmillionsofbrothersinwoemillionsofsistersnowheretogo'
